## Data Transformation Level 1 
##### Transforming data in Tournament Directory to dataframe and adding ingestion date.

##### Defining the schema ourself for the bating Dataframe

In [0]:
from pyspark.sql.types import StructType ,StructField,IntegerType,DoubleType,StringType, DateType

In [0]:
schema_tour = StructType([
    StructField("final_id", IntegerType(), False),
    StructField("date_final", StringType(), False),
    StructField("nbr_teams", IntegerType(), True),
    StructField("game_format", StringType(), True),
    StructField("venue", StringType(), True),
    StructField("venue_choice", StringType(), True),
    StructField("winner", StringType(), True),
    StructField("runner_up", StringType(), True),
    StructField("toss_win", StringType(), True),
    StructField("bowling_select", StringType(), True),
    StructField("player_top_scorer", StringType(), True),
    StructField("player_top_wkts",StringType(), True),
    StructField("player_series", StringType(), True),

])

In [0]:
df_tour_raw = spark.read.csv("/mnt/saprojectone/bronze-project1/tour",header="True", schema=schema_tour)

##### Adding Ingestion date column

In [0]:
from pyspark.sql.functions import current_timestamp

In [0]:
df_tour_t1 = df_tour_raw.withColumn("ingestion_date",current_timestamp())

In [0]:
from pyspark.sql.functions import to_date, to_timestamp

Need to add the below statement as the type of datetime in the raw data is not recognised in the new spark versions. Othherwise use regex to remove the day from the data and then call to_date.

In [0]:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

In [0]:
df_tour_t2 = df_tour_t1.withColumn("date_final", to_date(df_tour_t1["date_final"], "EEEE, MMMM dd, yyyy"))


##### Copying the data in the Silver container for 2nd level of transformation

In [0]:
df_tour_t2.write.mode("overwrite").format("delta").save("/mnt/saprojectone/silver-project1/tour")